In [1]:
import pandas as pd
from datasets import load_dataset
import os
from openai import OpenAI


# Import dataset from HuggingFace

In [26]:
dataset = load_dataset('go_emotions', 'raw')

dataset

In [29]:
df = dataset['train'].to_pandas()
# Remove unnecessary columns.
# All records have example_very_unclear = False
df = df.drop(['id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear'], axis=1)

df

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211220,Everyone likes [NAME].,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
211221,Well when you’ve imported about a gazillion of...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211222,That looks amazing,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211223,The FDA has plenty to criticize. But like here...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# From go_emotions README
label_encoding = {  '0': 'admiration',      '1': 'amusement',   '2': 'anger',           '3': 'annoyance',   '4': 'approval',    
                    '5': 'caring',          '6': 'confusion',   '7': 'curiosity',       '8': 'desire',      '9': 'disappointment',  
                    '10': 'disapproval',    '11': 'disgust',    '12': 'embarrassment',  '13': 'excitement', '14': 'fear',   
                    '15': 'gratitude',      '16': 'grief',      '17': 'joy',            '18': 'love',       '19': 'nervousness',    
                    '20': 'optimism',       '21': 'pride',      '22': 'realization',    '23': 'relief',     '24': 'remorse',    
                    '25': 'sadness',        '26': 'surprise',   '27': 'neutral' }

inverse_encoding = {}
for key, value in label_encoding.items():
    inverse_encoding[value] = key

# OpenAI ChatGPT 3.5

In [13]:
api_key = os.environ.get('OPENAI_API_KEY')
print(api_key)

None
